# Considerazioni
Non sembra valga la pena usare PlBe al posto di at_most_k_seq
Inoltre, ci mette 2 minuti e 20 rispetto ai 40secondi impiegati dal third attempt, in cui utilizzo diverse opzioni del sat solver s.sat()

In [1]:
from itertools import combinations
from z3 import *
import math
from utils import print_weekly_schedule
import time

# --- 1. Encoding Naive/Pairwise (NP) ---
def at_least_one_np(bool_vars):
    if not bool_vars: return BoolVal(False)
    return Or(bool_vars)

def at_most_one_np(bool_vars):
    if len(bool_vars) < 2: return []
    return [Not(And(bool_vars[i], bool_vars[j])) for i in range(len(bool_vars) - 1) for j in range(i+1, len(bool_vars))]

def exactly_one_np(bool_vars, name=''):
    return And(*at_most_one_np(bool_vars), at_least_one_np(bool_vars))


# --- 2. Binary Encoding (BW) ---
def toBinary(num, length = None):
    num_bin = bin(num).split("b")[-1]
    if length:
        return "0"*(length - len(num_bin)) + num_bin
    return num_bin
    
def at_least_one_bw(bool_vars):
    return at_least_one_np(bool_vars)

def at_most_one_bw(bool_vars, name=''):
    constraints = []
    n = len(bool_vars)
    if n == 0: return BoolVal(True)
    
    m = math.ceil(math.log2(n))
    r = [Bool(f"r_{name}_{i}") for i in range(m)]
    binaries = [toBinary(idx, m) for idx in range(n)]
    
    for i in range(n):
        phi_parts = []
        for j in range(m):
            if binaries[i][j] == "1":
                phi_parts.append(r[j])
            else:
                phi_parts.append(Not(r[j]))
        constraints.append(Or(Not(bool_vars[i]), And(*phi_parts)))

    return And(constraints)

def exactly_one_bw(bool_vars, name=''):
    return And(at_least_one_bw(bool_vars), at_most_one_bw(bool_vars, name))


# --- 3. Sequential Encoding (SEQ for k=1) ---
def at_least_one_seq(bool_vars):
    return at_least_one_np(bool_vars)

def at_most_one_seq(bool_vars, name=''):
    constraints = []
    n = len(bool_vars)
    if n == 0: return BoolVal(True)
    if n == 1: return BoolVal(True) 

    s = [Bool(f"s_{name}_{i}") for i in range(n - 1)]

    constraints.append(Or(Not(bool_vars[0]), s[0]))
    constraints.append(Or(Not(bool_vars[n-1]), Not(s[n-2])))
    
    for i in range(1, n - 1):
        constraints.append(Or(Not(bool_vars[i]), s[i]))
        constraints.append(Or(Not(bool_vars[i]), Not(s[i-1])))
        constraints.append(Or(Not(s[i-1]), s[i]))
    
    return And(constraints)

def exactly_one_seq(bool_vars, name=''):
    return And(at_least_one_seq(bool_vars), at_most_one_seq(bool_vars, name))


# --- 4. Hierarchical Encoding (HE) ---
def at_least_one_he(bool_vars):
    return at_least_one_np(bool_vars)

def at_most_one_he(bool_vars, name=''):
    n = len(bool_vars)
    if n == 0: return BoolVal(True)
    if n <= 4:
        return And(*at_most_one_np(bool_vars))
    
    y = Bool(f"y_{name}") 
    
    c1_list = at_most_one_np(bool_vars[:3] + [y]) 
    c2_boolref = at_most_one_he(bool_vars[3:] + [Not(y)], name + "_") 

    return And(*c1_list, c2_boolref)

def exactly_one_he(bool_vars, name=''):
    return And(at_most_one_he(bool_vars, name), at_least_one_he(bool_vars))


# --- 5. General K-Encoding (NP - Direct Encoding) ---
def at_least_k_np(bool_vars, k, name = ""):
    return at_most_k_np([Not(var) for var in bool_vars], len(bool_vars)-k, name)

def at_most_k_np(bool_vars, k, name = ""):
    return And([Or([Not(x) for x in X]) for X in combinations(bool_vars, k + 1)])

def exactly_k_np(bool_vars, k, name = ""):
    return And(at_most_k_np(bool_vars, k, name), at_least_k_np(bool_vars, k, name))


# --- 6. General K-Encoding (SEQ - Sequential Counter Encoding) ---
def at_most_k_seq(bool_vars, k, name=''):
    constraints = []
    n = len(bool_vars)
    
    if n == 0: return BoolVal(True)
    if k == 0: return And([Not(v) for v in bool_vars])
    if k >= n: return BoolVal(True)

    s = [[Bool(f"s_{name}_{i}_{j}") for j in range(k)] for i in range(n)] 
    
    # Inizializzazione per il primo elemento (bool_vars[0])
    constraints.append(Or(Not(bool_vars[0]), s[0][0])) 
    for j in range(1, k):
        constraints.append(Not(s[0][j]))

    # Propagazione per gli elementi successivi (i da 1 a n-1)
    for i in range(1, n):
        constraints.append(Or(Not(s[i-1][0]), s[i][0]))
        constraints.append(Or(Not(bool_vars[i]), s[i][0]))

        for j in range(1, k):
            constraints.append(Or(Not(s[i-1][j]), s[i][j]))
            constraints.append(Or(Not(bool_vars[i]), Not(s[i-1][j-1]), s[i][j]))
        
        # Vincolo "Al più k" finale
        constraints.append(Or(Not(bool_vars[i]), Not(s[i-1][k-1])))

    return And(constraints)

def at_least_k_seq(bool_vars, k, name=''):
    return at_most_k_seq([Not(var) for var in bool_vars], len(bool_vars)-k, name + "_neg")

def exactly_k_seq(bool_vars, k, name=''):
    return And(at_most_k_seq(bool_vars, k, name), at_least_k_seq(bool_vars, k, name))

# --- Funzione principale per la risoluzione del problema STS ---
def sts_new_mine(n, exactly_one, at_most_k, timeout=None):
    NUM_TEAMS = n
    NUM_WEEKS = NUM_TEAMS - 1
    NUM_PERIODS_PER_WEEK = n // 2
    
    if NUM_TEAMS % 2 != 0:
        print(f"Errore: Il numero di squadre (n={NUM_TEAMS}) deve essere un numero pari.")
        return None, None, None

    s = Solver()
    if timeout:
        s.set("timeout", timeout)
        print(f"  Timeout del solver impostato a {timeout} ms.")
    
    games_vars = {}
    for i in range(NUM_TEAMS):
        for j in range(NUM_TEAMS):
            if i != j:
                for w in range(NUM_WEEKS):
                    for p in range(NUM_PERIODS_PER_WEEK):
                        games_vars[(i, j, w, p)] = Bool(f'g_{i+1}_{j+1}_{w+1}_{p+1}')
                        
    # --- Vincoli ---
    # 1. Ogni coppia di squadre gioca esattamente una volta (casa o trasferta)
    for i in range(NUM_TEAMS):
        for j in range(i+1, NUM_TEAMS):
            i_j_all_possible_matches = []
            for w in range(NUM_WEEKS):
                for p in range(NUM_PERIODS_PER_WEEK):
                    i_j_all_possible_matches.append(games_vars[(i, j, w, p)])
                    i_j_all_possible_matches.append(games_vars[(j, i, w, p)])
            s.add(exactly_one(i_j_all_possible_matches, f"pair_once_{i}_{j}"))
    
            
    # 2. Ogni squadra gioca esattamente una volta a settimana
    for i in range(NUM_TEAMS):
        for w in range(NUM_WEEKS):
            once_week_game = []
            for j in range(NUM_TEAMS):
                if i == j: continue
                for p in range(NUM_PERIODS_PER_WEEK):
                    once_week_game.append(games_vars[(i,j,w,p)])
                    once_week_game.append(games_vars[(j,i,w,p)])
            s.add(exactly_one(once_week_game, f"team_once_week_{i}_{w}"))
            
    # 3. Ogni squadra gioca al massimo due volte nello stesso periodo (es. 'periodo 1' di diverse settimane)
    for i in range(NUM_TEAMS):
        for p in range(NUM_PERIODS_PER_WEEK):
            num_games_period = []
            for j in range(NUM_TEAMS):
                if i == j: continue
                for w in range(NUM_WEEKS):
                    num_games_period.append(games_vars[(i,j,w,p)])
                    num_games_period.append(games_vars[(j,i,w,p)])
            #s.add(at_most_k(num_games_period, 2, f"team_at_most_2_period_{i}_{p}"))
            s.add(PbLe([(var, 1) for var in num_games_period], 2))
            
    # 4. In ogni periodo di ogni settimana si gioca esattamente una partita
    for w in range(NUM_WEEKS):
        for p in range(NUM_PERIODS_PER_WEEK):
            game_in_period = []
            for i in range(NUM_TEAMS):
                for j in range(i+1, NUM_TEAMS):
                    game_in_period.append(games_vars[(i,j,w,p)])
                    game_in_period.append(games_vars[(j,i,w,p)])
            s.add(exactly_one(game_in_period, f"slot_one_game_{w}_{p}"))
            
            
    # Symmetry Breaking (facoltativo, può aiutare il solver)
    if NUM_TEAMS >= 2 and NUM_WEEKS >= 1 and NUM_PERIODS_PER_WEEK >= 1:
        s.add(games_vars[(0, 1, 0, 0)])
    
    start_solve_time = time.time()
    check_result = s.check()
    elapsed_solve_time = time.time() - start_solve_time
    
    print(f"  Solver ha restituito: {check_result} dopo {elapsed_solve_time:.2f} secondi.")

    if check_result == sat:
        print("Solver found a SAT model!")
        m = s.model()
        solution_schedule = []
        for (i, j, w, p), var in games_vars.items():
            if m.evaluate(var):
                solution_schedule.append((i+1, j+1, w+1, p+1))
        print(f"Number of active matches in solution: {len(solution_schedule)}")
        return solution_schedule, games_vars, m
    else:
        print("No solution found.")
        return None, None, None


# --- Blocco principale per l'esecuzione del test ---
if __name__ == '__main__':
    # --- SCEGLI QUI GLI ENCODING DA USARE ---
    # Assegna le funzioni desiderate a queste variabili globali
    
    exactly_one = exactly_one_he 
    at_most_k = at_most_k_seq   

    n_test = 12
    
    test_timeout_ms = 5 * 60 * 1000 

    print(f"Sto usando exactly_one: {exactly_one.__name__}, e at_most_k: {at_most_k.__name__}")
    print(f"Tentativo di risolvere STS per {n_test} squadre...")
    
    match_list, match_dict, model = sts_new_mine(n_test, exactly_one, at_most_k, timeout=test_timeout_ms)

    if match_list:
        print_weekly_schedule(match_list, n_test)

Sto usando exactly_one: exactly_one_he, e at_most_k: at_most_k_seq
Tentativo di risolvere STS per 12 squadre...
  Timeout del solver impostato a 300000 ms.
  Solver ha restituito: sat dopo 135.65 secondi.
Solver found a SAT model!
Number of active matches in solution: 66

--- SCHEDULE DEL TORNEO ---
Numero di squadre: 12
Numero di settimane: 11
Periodi per settimana: 6
---------------------------

Settimana 1:
  Periodo 1: Squadra 1 (Casa) vs Squadra 2 (Trasferta)
  Periodo 2: Squadra 6 (Casa) vs Squadra 10 (Trasferta)
  Periodo 3: Squadra 8 (Casa) vs Squadra 11 (Trasferta)
  Periodo 4: Squadra 4 (Casa) vs Squadra 7 (Trasferta)
  Periodo 5: Squadra 12 (Casa) vs Squadra 9 (Trasferta)
  Periodo 6: Squadra 5 (Casa) vs Squadra 3 (Trasferta)

Settimana 2:
  Periodo 1: Squadra 7 (Casa) vs Squadra 9 (Trasferta)
  Periodo 2: Squadra 3 (Casa) vs Squadra 12 (Trasferta)
  Periodo 3: Squadra 6 (Casa) vs Squadra 1 (Trasferta)
  Periodo 4: Squadra 11 (Casa) vs Squadra 4 (Trasferta)
  Periodo 5: Squa